Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 файлам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1 (3 балла)

1. Возьмите 10 000 случайных строк из таблицы sales_train и найдите в них долю товаров из категории 37. 
2. Найдите самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center shopping center за 2014 год.
3. Есть ли товар, который продается успешнее всех остальных (по ежемесячному количеству) во всех магазинах?

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# загрузка данных из файлов
sales_data = pd.read_csv("/content/drive/MyDrive/sales_train.csv")
items_data = pd.read_csv("/content/drive/MyDrive/items.csv")
categories_data = pd.read_csv("/content/drive/MyDrive/item_categories.csv")

# название столбца `category_id` в таблице `categories_data` 
# отличается от названия этого столбца в таблице `merged_data`
categories_data.rename(columns={'item_category_id':'category_id'}, inplace = True)

# выбор 10000 случайных строк из таблицы sales_data
random_sales_data = sales_data.sample(n=10000)

# объединение таблиц для получения информации о товаре и его категории
merged_data_items = pd.merge(random_sales_data, items_data, on='item_id')
merged_data = pd.merge(merged_data_items, categories_data, on='category_id')

# подсчет количества товаров из категории 37
category37_items_count = merged_data[merged_data["category_id"]==37]["item_id"].count()

# подсчет доли товаров из категории 37
fraction = category37_items_count / 10000 * 100

print("Доля товаров из категории 37: ", fraction, "%")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Доля товаров из категории 37:  6.52 %


In [ ]:
shops_data = pd.read_csv('/content/drive/MyDrive/shops.csv')
sales_data['date'] = pd.to_datetime(sales_data['date'])
# Фильтр по магазину и году
sales_spb = sales_data[(sales_data.shop_id == 30) & (sales_data.date.dt.year == 2014)]

# Объединение таблиц
sales_items = pd.merge(sales_spb, items_data, on='item_id')
sales_items_categories = pd.merge(sales_items, categories_data, on='category_id')
sales_items_categories_shops = pd.merge(sales_items_categories, shops_data, on='shop_id')

# Группировка по дням, суммирование количества проданных товаров и сортировка по убыванию
sales_per_day = sales_items_categories_shops.groupby('date')['item_cnt_day'].sum().sort_values(ascending=False)

# Вывод результатов
print(sales_per_day.index[0].date())

2014-12-31


In [ ]:
# объединяем данные о продажах с информацией о товарах и категориях
sales = sales_data.merge(items_data[['item_id', 'category_id']], on='item_id')
sales = sales.merge(categories_data, on='category_id')
sales = sales.merge(shops_data, on='shop_id')

# группируем по месяцам, товарам и магазинам
sales_by_month_item_shop = sales.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].sum().reset_index()

# группируем по месяцам и товарам, чтобы найти наиболее продаваемый товар каждый месяц
sales_by_month_item = sales_by_month_item_shop.groupby(['date_block_num', 'item_id'])['item_cnt_day'].sum().reset_index()
max_sales_by_month_item = sales_by_month_item.groupby('date_block_num')['item_cnt_day'].max().reset_index()

# находим id товаров с максимальным количеством продаж каждый месяц
top_items = []
for i in max_sales_by_month_item.index:
    max_sales = max_sales_by_month_item.loc[i, 'item_cnt_day']
    month = max_sales_by_month_item.loc[i, 'date_block_num']
    item_id = sales_by_month_item.loc[(sales_by_month_item['date_block_num'] == month) & 
                                      (sales_by_month_item['item_cnt_day'] == max_sales), 'item_id'].values[0]
    top_items.append(item_id)

# находим информацию о каждом из найденных товаров
top_items_info = items_data.loc[items_data['item_id'].isin(top_items), ['item_id', 'item_name']]
top_items_info['total_sales'] = max_sales_by_month_item['item_cnt_day']

# выводим информацию о товарах с наибольшим количеством продаж
print('Топ-товары:')
top_items_info

Топ-товары:


,item_id,item_name,total_sales
2445,2445,"Crysis 3 [PC, Russian version]",NaN
3731,3731,"Grand Theft Auto V [PC, Russian subtitles]",NaN
6738,6738,"StarCraft II: Heart of the Swarm (add-on) [PC,...",NaN
16450,16450,CLOUD ATLAS,NaN
20949,20949,Corporate package T-shirt 1C Interest white (3...,NaN
21376,21376,HOBBIT: AN UNDETALIZED JOURNEY (2DVD),NaN


# Задание 2 (2 балла)

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
# слияние таблиц по идентификаторам товаров и категорий
sales = pd.merge(sales_data, items_data, on='item_id')
sales = pd.merge(sales, categories_data, on='category_id')

# группировка данных по идентификаторам товаров и названиям
sales_grouped = sales.groupby(['item_id', 'item_name'])['item_cnt_day'].sum().reset_index()

# сортировка по убыванию и выбор топ-10
top_10_items = sales_grouped.sort_values('item_cnt_day', ascending=False).head(10)

top_10_items

,item_id,item_name,item_cnt_day
20602,20949,Corporate package T-shirt 1C Interest white (3...,187642.0
2749,2808,"Diablo III [PC, Jewel, Russian version]",17245.0
3654,3732,"Grand Theft Auto V [PS3, Russian subtitles]",16642.0
17418,17717,Acceptance of funds for 1C-Online,15830.0
5717,5822,Playstation Store replenishment of wallet: Pay...,14515.0
3656,3734,"Grand Theft Auto V [Xbox 360, Russian subtitles]",11688.0
6543,6675,Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...,10289.0
3653,3731,"Grand Theft Auto V [PC, Russian subtitles]",10099.0
1814,1855,"Battlefield 4 [PC, Russian version]",10032.0
16493,16787,"Some of us [PS3, Russian version]",9227.0


In [ ]:
years = [2013, 2014, 2015]
# 'date' column is not recognized as a string type
sales_data['date'] = sales_data['date'].astype(str)
for year in years:
    top_10_items = top_10_items[top_10_items['item_id'].isin(sales_data[sales_data['date'].str.contains(str(year))]['item_id'].unique())]
top_10_items

,item_id,item_name,item_cnt_day
20602,20949,Corporate package T-shirt 1C Interest white (3...,187642.0
2749,2808,"Diablo III [PC, Jewel, Russian version]",17245.0
3654,3732,"Grand Theft Auto V [PS3, Russian subtitles]",16642.0
17418,17717,Acceptance of funds for 1C-Online,15830.0
5717,5822,Playstation Store replenishment of wallet: Pay...,14515.0
3656,3734,"Grand Theft Auto V [Xbox 360, Russian subtitles]",11688.0
6543,6675,Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...,10289.0
1814,1855,"Battlefield 4 [PC, Russian version]",10032.0
16493,16787,"Some of us [PS3, Russian version]",9227.0


# Задание 3 (3 балла)

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

*Если таких категорий несколько, выведите все.*

In [ ]:
min_category_id = sales_data.join(items_data, on='item_id', lsuffix='_left').join(categories_data, on='category_id', lsuffix='_left') \
                 .groupby('category_id').agg({'item_cnt_day': 'sum'}) \
                 .idxmin()[0]
# print(item_categories.columns)                 
min_category_name = categories_data[categories_data['category_id'] == min_category_id]['item_category_name'].iloc[0]
print(f'Категория товаров с минимальным числом покупок: {min_category_id}, {min_category_name}')

Категория товаров с минимальным числом покупок: 10,  Game consoles - PS2


In [ ]:
max_category_id = sales_data.join(items_data, on='item_id', lsuffix='_left').join(categories_data, on='category_id', lsuffix='_left') \
                 .groupby('category_id').agg({'item_cnt_day': 'sum'}) \
                 .idxmax()[0]
max_category_name = categories_data[categories_data['category_id'] == max_category_id]['item_category_name'].iloc[0]
print(f'Категория товаров с максимальным числом покупок: {max_category_id}, {max_category_name}')

Категория товаров с максимальным числом покупок: 40,  Cinema - DVD


# Задание 4 (3 балла)

1. Для категорий из задания 3 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами. Оставьте в таблице только те товары, среднее количество продаж которых строго больше 100.
2. Найдите месяц с максимальными продажами для каждого товара из пункта 1.


In [ ]:
# объединение таблиц
sales_items = pd.merge(sales_data, items_data, on='item_id')
sales_items_categories = pd.merge(sales_items, categories_data, on='category_id')
sales_items_categories_shops = pd.merge(sales_items_categories, shops_data, on='shop_id')

# выборка данных для категорий 10 и 40 из здания 3
categories = sales_items_categories_shops[sales_items_categories_shops['category_id'].isin([10, 40])]

# среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами
top5_shops = categories.groupby(['category_id', 'item_id', 'shop_id'])['item_cnt_day'].sum().reset_index()
top5_shops = top5_shops.groupby(['category_id', 'item_id'])[['item_cnt_day']].sum().reset_index()
top5_shops = top5_shops.sort_values(by='item_cnt_day', ascending=False).groupby('category_id').head(5)

# оставляем только товары, среднее количество продаж которых строго больше 100
top5_shops = top5_shops.groupby(['category_id', 'item_id']).filter(lambda x: x['item_cnt_day'].mean() > 100)

# месяц с максимальными продажами для каждого товара
max_sales_month = categories.groupby(['category_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().reset_index()
max_sales_month = max_sales_month.sort_values(by='item_cnt_day', ascending=False).groupby(['category_id', 'item_id']).head(1)[['category_id', 'item_id', 'date_block_num']]

print('Топ 5 товаров категории 10 и среднее количество продаж:')
top5_shops[top5_shops['category_id'] == 10][['category_id', 'item_id', 'item_cnt_day']]

Топ 5 товаров категории 10 и среднее количество продаж:


,category_id,item_id,item_cnt_day


In [ ]:
print('Топ 5 товаров категории 40 и среднее количество продаж:')
top5_shops[top5_shops['category_id'] == 40][['category_id', 'item_id', 'item_cnt_day']]

Топ 5 товаров категории 40 и среднее количество продаж:


,category_id,item_id,item_cnt_day
1059,40,10298,3563.0
4624,40,21404,3525.0
4645,40,21440,3402.0
2219,40,13811,2664.0
1572,40,11496,2588.0


In [ ]:
print(' Месяц с максимальными продажами для каждого товара из категорий 10 и 40: \n')
max_sales_month

 Месяц с максимальными продажами для каждого товара из категорий 10 и 40: 



,category_id,item_id,date_block_num
23736,40,13811,5
49888,40,21376,3
11155,40,10298,10
31787,40,16450,0
45063,40,19196,2
...,...,...,...
31223,40,15800,0
31219,40,15792,0
30843,40,15718,10
4016,40,8642,6


# Задание 5 (3 балла)

1. Для категорий из задания 3 найдите среднюю и медианную цену по категории в пределах каждого месяца. Выведите топ-10 товаров по медиане из каждой категории.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
# объединение таблиц
sales_items = pd.merge(sales_data, items_data, on='item_id')
sales_items_categories = pd.merge(sales_items, categories_data, on='category_id')

# фильтрация данных по категориям 10 и 40
categories_10_40 = sales_items_categories[(sales_items_categories['category_id'] == 10) | (sales_items_categories['category_id'] == 40)]

# группировка данных по месяцу и категории, вычисление средней и медианной цены
grouped = categories_10_40.groupby(['date_block_num', 'category_id']).agg({'item_price': ['mean', 'median']})

# сброс мультииндекса столбцов и переименование столбцов
grouped.columns = grouped.columns.map('_'.join)
grouped = grouped.reset_index().rename(columns={'item_price_mean': 'mean_price', 'item_price_median': 'median_price'})

# топ-10 товаров по медианной цене в каждой категории
top_10_10 = grouped[grouped['category_id'] == 10].sort_values(by='median_price', ascending=False).head(10)
top_10_40 = grouped[grouped['category_id'] == 40].sort_values(by='median_price', ascending=False).head(10)

print('Топ-10 товаров в категории 10 по средней цене:')
top_10_10[['date_block_num', 'category_id', 'mean_price', 'median_price']]

Топ-10 товаров в категории 10 по средней цене:


,date_block_num,category_id,mean_price,median_price
3,3,10,1998.0,1998.0


In [ ]:
print('Топ-10 товаров в категории 40 по средней цене:')
top_10_40[['date_block_num', 'category_id', 'mean_price', 'median_price']]

Топ-10 товаров в категории 40 по средней цене:


,date_block_num,category_id,mean_price,median_price
29,28,40,286.508608,299.0
28,27,40,281.630704,299.0
15,14,40,262.435981,251.0
34,33,40,281.000153,249.0
16,15,40,268.408070,249.0
33,32,40,279.393699,249.0
32,31,40,273.242103,249.0
30,29,40,279.142252,249.0
14,13,40,261.256517,248.0
21,20,40,265.388479,240.0


In [ ]:
sales_data['date'] = pd.to_datetime(sales_data['date'])

# вычисление года и месяца продажи
sales_data['year'] = sales_data['date'].dt.year
sales_data['month'] = sales_data['date'].dt.month

# группировка данных по году, месяцу и магазину, вычисление средней цены
grouped = sales_data.groupby(['year', 'month', 'shop_id']).agg({'item_price': 'mean'}).reset_index()

# добавление столбцов со средней ценой предыдущего и следующего месяца
grouped['prev_month_mean'] = grouped.groupby(['shop_id'])['item_price'].shift(1)
grouped['next_month_mean'] = grouped.groupby(['shop_id'])['item_price'].shift(-1)

# фильтрация данных по месяцам, в которых средняя цена меньше, чем в предыдущем и следующем месяце
filtered = grouped[(grouped['item_price'] < grouped['prev_month_mean']) & (grouped['item_price'] < grouped['next_month_mean'])]

print('Месяцы с более низкой средней ценой по сравнению с предыдущим и следующим месяцами:')
filtered[['year', 'month', 'shop_id', 'item_price', 'prev_month_mean', 'next_month_mean']]

Месяцы с более низкой средней ценой по сравнению с предыдущим и следующим месяцами:


,year,month,shop_id,item_price,prev_month_mean,next_month_mean
53,2013,2,3,724.225225,735.678370,758.263125
55,2013,2,5,622.936012,642.917822,661.464245
60,2013,2,12,1247.255632,1251.325630,1341.278633
64,2013,2,16,652.050130,697.110518,675.302259
66,2013,2,18,902.085154,988.646640,903.885642
...,...,...,...,...,...,...
1775,2015,11,39,1034.427313,1417.116436,1181.688742
1777,2015,11,41,818.984000,1221.274169,1092.817500
1779,2015,11,43,2268.676471,2747.182927,2328.653846
1781,2015,11,45,794.947791,1109.201198,1005.200000
